### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/410_TROCAS7_Fusion_Apr2021_PEAKS_135/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/410_TROCAS7_Fusion_Apr2021_PEAKS_135


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 410_TROCAS7_Fusion_Apr2021_PEAKS_135_peptide.csv
 410_TROCAS7_Fusion_Apr2021_PEAKS_135_protein.csv
 410_TROCAS7_Fusion_Apr2021_PEAKS_135_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup410 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/410_TROCAS7_Fusion_Apr2021_PEAKS_135/410_TROCAS7_Fusion_Apr2021_PEAKS_135_peptide.csv")

# remove redundant rows
peaksdb410 = pd.DataFrame.drop_duplicates(peaksdbdup410)

print(peaksdb410.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb410.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb410['#Spec']
del peaksdb410['Accession']
del peaksdb410['PTM']
del peaksdb410['AScore']

mean_length = peaksdb410['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup410))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb410))

#look at the dataframe
peaksdb410.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-410-SMCP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-410-SMCP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.415094339622641
# redundant peaksdb peptides in combined dataframe 106
# nonredundant peaksdb peptides in combined dataframe 106


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,73.38,1881.8350,18,1.3,941.9260,92.34,921000.0,12,15036,20210406_Trocas7_656_SMCP410_DDA_120min_1.raw
1,SSGSSYPSLLQC(+57.02)LK,59.05,1525.7446,14,0.7,763.8801,100.43,186000.0,12,16406,20210406_Trocas7_656_SMCP410_DDA_120min_1.raw
2,IDVLEGNEQFINAAK,49.97,1659.8468,15,0.6,830.9312,95.56,364000.0,12,15587,20210406_Trocas7_656_SMCP410_DDA_120min_1.raw
3,IITHPNFNGNTLDND,43.23,1683.7853,15,-0.8,842.8992,73.86,2250000.0,12,12174,20210406_Trocas7_656_SMCP410_DDA_120min_1.raw
4,LGEHNIDVLEGNEQ,42.96,1565.7322,14,0.8,783.8740,76.16,821000.0,12,12541,20210406_Trocas7_656_SMCP410_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb410['A'] = peaksdb410['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb410['C'] = peaksdb410['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb410['D'] = peaksdb410['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb410['E'] = peaksdb410['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb410['F'] = peaksdb410['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb410['G'] = peaksdb410['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb410['H'] = peaksdb410['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb410 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb410['I'] = peaksdb410['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb410['K'] = peaksdb410['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb410['L'] = peaksdb410['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb410['M'] = peaksdb410['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb410['N'] = peaksdb410['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb410['P'] = peaksdb410['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb410['Q'] = peaksdb410['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb410['R'] = peaksdb410['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb410['S'] = peaksdb410['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb410['T'] = peaksdb410['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb410['V'] = peaksdb410['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb410['W'] = peaksdb410['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb410['Y'] = peaksdb410['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb410['c-carb'] = peaksdb410['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb410['m-oxid'] = peaksdb410['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb410['n-deam'] = peaksdb410['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb410['q-deam'] = peaksdb410['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb410['stripped_peptide'] = peaksdb410['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb410['stripped_length'] = peaksdb410['stripped_peptide'].apply(len)

peaksdb410['NAAF_num.'] = peaksdb410['Area'] / peaksdb410['stripped_length']

# total the number of modifications in sequence
peaksdb410['ptm-total'] = peaksdb410['c-carb'] + peaksdb410['m-oxid'] + peaksdb410['n-deam'] + peaksdb410['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb410['stripped_IL']= peaksdb410['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB.csv")

# check out the results
peaksdb410.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,73.38,1881.8350,18,1.3,941.9260,92.34,921000.0,12,15036,...,0,2,0,0,0,SCLISGWGNTK,11,83727.272727,2,SCLLSGWGNTK
1,SSGSSYPSLLQC(+57.02)LK,59.05,1525.7446,14,0.7,763.8801,100.43,186000.0,12,16406,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,13285.714286,1,SSGSSYPSLLQCLK
2,IDVLEGNEQFINAAK,49.97,1659.8468,15,0.6,830.9312,95.56,364000.0,12,15587,...,0,0,0,0,0,IDVLEGNEQFINAAK,15,24266.666667,0,LDVLEGNEQFLNAAK
3,IITHPNFNGNTLDND,43.23,1683.7853,15,-0.8,842.8992,73.86,2250000.0,12,12174,...,0,0,0,0,0,IITHPNFNGNTLDND,15,150000.000000,0,LLTHPNFNGNTLDND
4,LGEHNIDVLEGNEQ,42.96,1565.7322,14,0.8,783.8740,76.16,821000.0,12,12541,...,0,0,0,0,0,LGEHNIDVLEGNEQ,14,58642.857143,0,LGEHNLDVLEGNEQ


In [6]:
# keep only stripped peptide I/L and NAAF
db_410 = peaksdb410[['stripped_IL', 'Area', 'NAAF_num.']]

db_410.set_index('stripped_IL')

# write modified dataframe to new txt file
db_410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/410A_MS_T24_GD_PDB_ILnaafs.csv")

db_410.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,921000.0,83727.272727
1,SSGSSYPSLLQCLK,186000.0,13285.714286
2,LDVLEGNEQFLNAAK,364000.0,24266.666667
3,LLTHPNFNGNTLDND,2250000.0,150000.000000
4,LGEHNLDVLEGNEQ,821000.0,58642.857143


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep410moddup = peaksdb410[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep410dup = peaksdb410[["stripped_peptide"]]

# deduplicate both of these lists
pep410mod = pep410moddup.drop_duplicates()
pep410 = pep410dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 410:", len(pep410moddup))
print("Deduplicated modified PeaksDB peptides in 410:", len(pep410mod))
print("Total stripped PeaksDB peptides in 410:", len(pep410dup))
print("Deduplicated stripped PeaksDB peptides in 410:", len(pep410))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep410.head()

Total modified PeaksDB peptides in 410: 106
Deduplicated modified PeaksDB peptides in 410: 106
Total stripped PeaksDB peptides in 410: 106
Deduplicated stripped PeaksDB peptides in 410: 100


,stripped_peptide
0,SCLISGWGNTK
1,SSGSSYPSLLQCLK
2,IDVLEGNEQFINAAK
3,IITHPNFNGNTLDND
4,LGEHNIDVLEGNEQ


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb410['A'].sum(),
        'C': peaksdb410['C'].sum(),
        'D': peaksdb410['D'].sum(),
        'E': peaksdb410['E'].sum(),
        'F': peaksdb410['F'].sum(),
        'G': peaksdb410['G'].sum(),
        'H': peaksdb410['H'].sum(),
        'I': peaksdb410['I'].sum(),
        'K': peaksdb410['K'].sum(),
        'L': peaksdb410['L'].sum(),
        'M': peaksdb410['M'].sum(),
        'N': peaksdb410['N'].sum(),
        'P': peaksdb410['P'].sum(),
        'Q': peaksdb410['Q'].sum(),
        'R': peaksdb410['R'].sum(),
        'S': peaksdb410['S'].sum(),
        'T': peaksdb410['T'].sum(),
        'V': peaksdb410['V'].sum(),
        'W': peaksdb410['W'].sum(),
        'Y': peaksdb410['Y'].sum(),
        'c-carb': peaksdb410['c-carb'].sum(),
        'm-oxid': peaksdb410['m-oxid'].sum(),
        'n-deam': peaksdb410['n-deam'].sum(),
        'q-deam': peaksdb410['q-deam'].sum(),
        'Total area': peaksdb410['Area'].sum(),
        'Total length': peaksdb410['stripped_length'].sum()
       }

totalpeaksdb410 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb410['% C w/ carb'] = totalpeaksdb410['c-carb'] / totalpeaksdb410['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb410['% M w/ oxid'] = totalpeaksdb410['m-oxid'] / totalpeaksdb410['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb410['% N w/ deam'] = totalpeaksdb410['n-deam'] / totalpeaksdb410['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb410['% Q w/ deam'] = totalpeaksdb410['q-deam'] / totalpeaksdb410['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb410['NAAF denom.'] = totalpeaksdb410['Total area'] / totalpeaksdb410['Total length']

# write modified dataframe to new txt file
totalpeaksdb410.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_totals.csv")

totalpeaksdb410.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,78,23,46,80,31,98,22,76,53,124,...,7,14,17,311780920.0,1046,1.0,0.636364,0.145833,0.404762,298069.713193


In [9]:
# use the calculated NAAF factor (in totalpeaksdb410 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 298069.713193

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb410['NAAF factor'] = (peaksdb410['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb410_NAAF = peaksdb410[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb410_NAAF['A-NAAF20'] = peaksdb410_NAAF['A'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['C-NAAF20'] = peaksdb410_NAAF['C'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['D-NAAF20'] = peaksdb410_NAAF['D'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['E-NAAF20'] = peaksdb410_NAAF['E'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['F-NAAF20'] = peaksdb410_NAAF['F'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['G-NAAF20'] = peaksdb410_NAAF['G'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['H-NAAF20'] = peaksdb410_NAAF['H'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['I-NAAF20'] = peaksdb410_NAAF['I'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['K-NAAF20'] = peaksdb410_NAAF['K'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['L-NAAF20'] = peaksdb410_NAAF['L'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['M-NAAF20'] = peaksdb410_NAAF['M'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['N-NAAF20'] = peaksdb410_NAAF['N'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['P-NAAF20'] = peaksdb410_NAAF['P'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['Q-NAAF20'] = peaksdb410_NAAF['Q'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['R-NAAF20'] = peaksdb410_NAAF['R'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['S-NAAF20'] = peaksdb410_NAAF['S'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['T-NAAF20'] = peaksdb410_NAAF['T'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['V-NAAF20'] = peaksdb410_NAAF['V'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['W-NAAF20'] = peaksdb410_NAAF['W'] * peaksdb410['NAAF factor']
peaksdb410_NAAF['Y-NAAF20'] = peaksdb410_NAAF['Y'] * peaksdb410['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb410_NAAF['ccarb-NAAF20'] = peaksdb410_NAAF['c-carb'] * peaksdb410_NAAF['NAAF factor']
peaksdb410_NAAF['moxid-NAAF20'] = peaksdb410_NAAF['m-oxid'] * peaksdb410_NAAF['NAAF factor']
peaksdb410_NAAF['ndeam-NAAF20'] = peaksdb410_NAAF['n-deam'] * peaksdb410_NAAF['NAAF factor']
peaksdb410_NAAF['qdeam-NAAF20'] = peaksdb410_NAAF['q-deam'] * peaksdb410_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb410_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_naaf.csv")

peaksdb410_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.280898,3,2,0,1,0,3,0,1,...,0.0,0.561797,0.561797,0.000000,0.280898,0.000000,0.561797,0.0,0.0,0.0
1,SSGSSYPSLLQCLK,0.044573,0,1,0,0,0,1,0,0,...,0.0,0.222863,0.000000,0.000000,0.000000,0.044573,0.044573,0.0,0.0,0.0
2,IDVLEGNEQFINAAK,0.081413,2,0,1,2,1,1,0,2,...,0.0,0.000000,0.000000,0.081413,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,IITHPNFNGNTLDND,0.503238,0,0,2,0,1,1,1,2,...,0.0,0.000000,1.006476,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,LGEHNIDVLEGNEQ,0.196742,0,0,1,3,0,2,1,1,...,0.0,0.000000,0.000000,0.196742,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [10]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb410 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb410_NAAF['NAAF factor'].sum(),
        'A': peaksdb410_NAAF['A-NAAF20'].sum(),
        'C': peaksdb410_NAAF['C-NAAF20'].sum(),
        'D': peaksdb410_NAAF['D-NAAF20'].sum(),
        'E': peaksdb410_NAAF['E-NAAF20'].sum(),
        'F': peaksdb410_NAAF['F-NAAF20'].sum(),
        'G': peaksdb410_NAAF['G-NAAF20'].sum(),
        'H': peaksdb410_NAAF['H-NAAF20'].sum(),
        'I': peaksdb410_NAAF['I-NAAF20'].sum(),
        'K': peaksdb410_NAAF['K-NAAF20'].sum(),
        'L': peaksdb410_NAAF['L-NAAF20'].sum(),
        'M': peaksdb410_NAAF['M-NAAF20'].sum(),
        'N': peaksdb410_NAAF['N-NAAF20'].sum(),
        'P': peaksdb410_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb410_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb410_NAAF['R-NAAF20'].sum(),
        'S': peaksdb410_NAAF['S-NAAF20'].sum(),
        'T': peaksdb410_NAAF['T-NAAF20'].sum(),
        'V': peaksdb410_NAAF['V-NAAF20'].sum(),
        'W': peaksdb410_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb410_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb410_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb410_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb410_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb410_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb410_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb410_NAAF['% C w/ carb.'] = totalpeaksdb410_NAAF['c-carb'] / totalpeaksdb410_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb410_NAAF['% M w/ oxid'] = totalpeaksdb410_NAAF['m-oxid'] / totalpeaksdb410_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb410_NAAF['% N w/ deam'] = totalpeaksdb410_NAAF['n-deam'] / totalpeaksdb410_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb410_NAAF['% Q w/ deam'] = totalpeaksdb410_NAAF['q-deam'] / totalpeaksdb410_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb410_NAAF['NAAF check'] = totalpeaksdb410_NAAF['NAAF'] / 298069.713193

# write modified dataframe to new txt file, same name + totals
totalpeaksdb410_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/410_MS_T24_GD_PDB/410A_MS_T24_GD_PDB_naaf_totals.csv")

totalpeaksdb410_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,124.472833,103.457841,4.08253,4.02554,5.365336,2.234716,9.626725,2.989574,36.824737,3.450799,...,1.587079,4.08253,0.132253,0.406751,0.669792,1.0,0.192173,0.014181,0.244521,0.000418
